# Assignment of peer feedback

This notebook contains the code used to determine which two other teams each team must give feed back to in the data project

In [ ]:
from github import Github # pip install PyGithub
import sys
import datetime
import pandas as pd
import numpy as np
import random
from collections import Counter

In [ ]:
%load_ext autoreload
%autoreload 2 
from groups_utils import load_repos

### Load all repositories in this year's class room

In [ ]:
class_name, all_repos, current_class, disregard_repos = load_repos()

### All teams

In [ ]:
teams = set()
i = 1
for repo in all_repos:
    if repo.name not in current_class: continue
    if repo.name[14:] in disregard_repos: 
        print(repo.name)
        continue 
    teams.add(repo.name[14:])
    i += 1

### Excel sheets

In [ ]:
random.seed(123)
for k in range(10):

    try:

        for projectname in ['inauguralproject', 'dataproject', 'modelproject']:

            # a. create data frame
            teams_df = pd.DataFrame(data=teams,columns=['team'])
            teams_df.sort_values(by='team',inplace=True)
            teams_df.reset_index(drop=True, inplace=True)
            teams_df['peer_group_1'] = ''
            teams_df['peer_group_2'] = ''

            # b. assign peers to teams
            N = len(teams_df.team.values)
            counter = [0 for _ in range(N)]
            assigned = []

            # for each team, loop over teams that have not already been assigned
            for i,_ in enumerate(teams_df.team):
                pop = [t for t in range(N) if (t != i) & (t not in assigned)]
                peers = random.sample(pop, 2)    
                for c in [0,1]:
                    teams_df.iloc[i, 1 + c] = teams_df.team[peers[c]]
                    counter[peers[c]] += 1
                    if counter[peers[c]] == 2:
                        assigned.append(peers[c])

            teams_df.to_excel(f'PeerAssignment_{projectname}.xlsx',index=False)

        print(f'attempt {k+1} was successful')
        break

    except Exception as e:

        print(f'attempt {k+1} failed: {e}')